# Introduction to Python and Natural Language Technologies
## Lecture 05, Data science, Handling text data, Pandas

March 09, 2021

Ádám Kovács

The aim of the laboratory is to give a brief introduction to the popular Python library "Pandas". During the exercises we are going to use an IMDB dataset.

* Pandas is a popular Data science library for handling structural data. It has a lot of built in functions which you can explore [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)
* Pandas is a great tool for doing Exploratory Data Analysis (EDA) and visualization on your data.
* Before doing any Machine Learning, you always need to spend time to get to know your data.
* Bash tools can be also used for quick data manipulations.

First import the relevant libraries, you might have to install some of them.

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install numpy

Download and extract the data with Python code:

In [ ]:
import os

data_dir = os.getenv("data")
if data_dir is None:
    data_dir = ""

ml_path = os.path.join(data_dir, "data.zip")

if not os.path.exists(ml_path):
    print("Download data")
    import urllib
    u = urllib.request.URLopener()
    u.retrieve("http://sandbox.hlt.bme.hu/~adaamko/data.zip", ml_path)

Extracting:

In [ ]:
unzip_path = os.path.join(data_dir, "data")

if not os.path.exists(unzip_path):
    print("Unzip data")
    from zipfile import ZipFile
    with ZipFile(ml_path) as myzip:
        myzip.extractall(data_dir)
        
data_dir = unzip_path

### Get the occurence of each country from the release_dates file

In [ ]:
!cut -f3 -d, data/release_dates.csv | sort | uniq -c | sort

### Count the most frequent words in the titles

In [ ]:
!cat data/titles.csv | sort | uniq -u | wc -l
!cut -f1 -d, data/titles.csv | tr [:blank:] \\n | sort | uniq -c | sort | tail -10

In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np

matplotlib.style.use('ggplot')
matplotlib.pyplot.rcParams['figure.figsize'] = (15, 3)
matplotlib.pyplot.rcParams['font.family'] = 'sans-serif'

## Loading datasets with pandas

The function `pd.read_table` is used to load tabular data. It contains a lot of [parameters](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html), with the file being the only required one.

In [ ]:
%%time
cast = pd.read_table('data/cast.csv')

In [ ]:
cast.head()

In [ ]:
cast = pd.read_table('data/cast.csv', sep=",", index_col=None)
cast.head()

In [ ]:
cast.tail(3)

In [ ]:
cast.title
cast["title"]
type(cast)

`pd.read_table` function returns a `DataFrame` object. Columns can be selected as attributes or by their names between parentheses. If we select only one column it will return a `Series` object.

In [ ]:
type(cast.title)

More than one column can be also selected:

In [ ]:
cast[["title", "year"]]

We can also select one column as a DataFrame, if we select that column as a list

In [ ]:
type(cast[["title"]])

A DataFrame can be indexed in two ways, using either the `loc[]` or the `iloc[]` method. `iloc` returns a row by its position and accepts only integer value, `loc` returns a row by its label, which can be string also.

In [ ]:
cast.iloc[1:6]

In [ ]:
cast.loc[1:6]

In [ ]:
cast.loc[cast.year == 1985]

We can also select columns with `iloc`

In [ ]:
cast.iloc[:, 2:5]

Rows and columns can be retrieved in any order.

In [ ]:
cast.iloc[[2,1,2], [1,1,3]]

Selecting one cell:

In [ ]:
cast.loc[0, "name"]

## First look on the data
Let's look what information can be easily obtained from the DataFrame.

Using the describe function we can acquire basic information about the rows: count, average, variance etc.

In [ ]:
cast.describe()

In [ ]:
cast.year.describe()

In [ ]:
cast.year.max()

select only the movies

In [ ]:
titles = cast[['title', 'year']].drop_duplicates().reset_index(drop=True)
titles.head()

In [ ]:
titles.sort_values("year").tail(3)

## Simple queries

In [ ]:
# What movies were released in 1956?
titles[titles.year == 1956]

In [ ]:
# How many movies?
len(titles[titles.year == 1956])

In [ ]:
# Querying the movies from the 90s

titles[(titles.year >= 1990) & (titles.year <= 1999)]

In [ ]:
# List the Star Wars movies

titles[titles.title == "Star Wars"]

In [ ]:
titles[titles.title.str.contains("Star Wars: ")]

In [ ]:
# Movies that fall between A and B in the alphabet?
titles[(titles.title >= 'A') & (titles.title <= 'B')]

In [ ]:
t = titles.copy()
t = t.set_index('title').sort_index()
t.loc['A':'B']

In [ ]:
# Oldest and newest movies
titles.iloc[titles.year.idxmax()]

In [ ]:
# Longest title
t = titles.copy()
t.iloc[titles.title.str.len().idxmax()]
#t[t.title.str.match("^The .*")].sort_values(by="year")
#t[t.title.str.match(r"^[a-zA-Z]+\s[a-zA-Z]+$")]
#t.title.str.replace(r"^[a-zA-Z]+\s[a-zA-Z]+$", "two words")

In [ ]:
titles.iloc[titles.title.str.len().idxmin()]

## Grouping and visualization
How many movies were released by year?

We can answer the question in two steps:

* grouping by years
* aggregate for each year

In [ ]:
titles.groupby('year').size().plot()

In [ ]:
titles.groupby('year').size().plot(kind="bar")

In [ ]:
titles[(titles.year > 1999) & (titles.year < 2010)].groupby('year').size().plot(kind='bar')

In [ ]:
cast.head()

In [ ]:
cast[cast.name == "Matthew McConaughey"].groupby("year").size()

In [ ]:
# Show a stacked diagram about the "n"(role number) values in the 90s
i = cast.set_index('name').sort_index()
i

In [ ]:
i.loc["Anthony Hopkins", ["year", "n"]].groupby("year").size()

In [ ]:
i.loc["Anthony Hopkins", ["year", "n"]].groupby("year").agg(["min", "mean", "max"]).plot(kind="area", stacked=True)

Operators are overloaded so expressions are done on every row

In [ ]:
cast.year + 4000

In [ ]:
t = titles.copy()
t["movie_token_len"] = t.title.apply(lambda x: len(x.split()))
t.head()

In [ ]:
t.groupby("year").mean().plot()

In [ ]:
# What are the 11 most common character names in movie history?

cast.character.value_counts().head(11)

In [ ]:
# Which ten people have appeared most often as “Himself” over the history of film?

c = cast.copy()
c = c[c.character == 'Himself']
c = c.groupby('name').size()
c.sort_values().tail(10)

In [ ]:
c = cast.groupby(["character", "type"]).size()
c

### How many actors are in the cast of each version of Hamlet?

In [ ]:
cast[cast.title.str.startswith("Hamlet")]

In [ ]:
cast_by_title_year = cast.set_index(['title', 'year']).sort_index()
cast_by_title_year

In [ ]:
cast_by_title_year.loc["Hamlet", 1910]

In [ ]:
# How many actors are in the cast of each version of Hamlet?

cast_by_title_year.loc["Hamlet": "Hamlet (Z"]
#cast_by_title_year.loc["Hamlet": "Hamlet (Z"].groupby(level=[0,1]).size()

In [ ]:
#    Build a dataframe with a row for each year with two columns:
#    the number of roles for actors in that year's films,
#    and the number of roles for actresses.

aa = cast[['year', 'type']].groupby(['year', 'type']).size()
aa.head()

In [ ]:
aa.unstack(level=1)

In [ ]:
aa = aa.unstack()
aa = aa.loc[1990:2010]
aa

The `stack` and `unstack` functions are used to reshape the data into a stacked/unstacked format. 
![stack](https://www.w3resource.com/w3r_images/pandas-dataframe-unstack-1.svg)

In [ ]:
aa.plot(kind='area')

## Release dates and merging

In [ ]:
release_dates = pd.read_csv('data/release_dates.csv', index_col=None,
                            parse_dates=['date'], infer_datetime_format=True)

In [ ]:
release_dates.head()

In [ ]:
release_dates.groupby(release_dates.date.dt.month).size().plot()
#release_dates.groupby(release_dates.date.dt.day).size().plot(kind="pie")

**We could ask the question of when X actors movies were released?**. 
For that we need to merge a subset of the cast table to the release_dates table

In [ ]:
rd = release_dates.set_index("title").sort_index()
rd.drop_duplicates()

In [ ]:
cast_leonardo = cast[cast.name == "Leonardo DiCaprio"]
cast_leonardo.head()

In [ ]:
cast_date_leo = pd.merge(cast_leonardo, rd, left_on="title", right_index=True)

In [ ]:
cast_date_leo.groupby("country").size().idxmax()

In [ ]:
cast_date_leo.groupby(cast_date_leo.date.dt.month).size().plot()

"Pivot": Reshape data (produce a “pivot” table) based on column values. Uses unique values from specified index / columns to form axes of the resulting DataFrame. 

In [ ]:
r = release_dates
r = r[r.title.str.startswith('Star Wars: Episode')]
r = r[r.country.str.startswith('H')]
r.head()

In [ ]:
piv = r.pivot('title', 'country', 'date')
#piv = r.pivot('country', 'title', 'date')

In [ ]:
piv.head()

#### Pandas has GUI support  (see https://pypi.org/project/pandasgui/)

In [ ]:
!pip install pandasgui

In [ ]:
from pandasgui import show

gui = show(titles.title)

# Exercises?

## 1. Display the cast of "The Lord of the Rings: The Fellowship of the Ring" in their correct "n"-value order, ignoring roles that did not earn a numeric "n" value.

## 2. What are the 10 most frequent roles that contains the word "teacher"?

## 3. List each of the characters that Jackie Chan has portrayed at least twice. 

## 4. Plot the difference between the number of actor roles each year and the number of actress roles each year over the history of film.

## 5. Make a bar plot of the day of the week on which movies with "Romance" in their title tend to be released in the USA.